In [6]:
import os
import datetime
from pymongo import MongoClient
from dotenv import load_dotenv
from bson import Decimal128

In [8]:
load_dotenv('./.env')
MONGODB_URI = os.environ['MONGODB_URI']
OPTIONS = os.environ['OPTIONS']

client = MongoClient(
    MONGODB_URI+OPTIONS)
db = client.WheatherDB
dados_metereologicos = db.dados_metereologicos
regiao = db.regiao

In [ ]:
results = []
print(dados_metereologicos.find(projection = {'_id': 0}))
for dados in dados_metereologicos.find(projection={'_id': 0, 'id_regiao': 0}):
    for key, value in dados.items():
        if (type(value) == Decimal128):
            dados[key] = float(str(dados[key]))
        elif (type(value) == datetime.datetime):
            dados[key] = dados[key].strftime('%d/%m/%y %H:%M:%S')
    results.append(dados)


In [ ]:
dados = [
    {'umidade': 80.0, 'temperatura': 20.0, 'dt_criacao': '16/09/23 01:11:18', 'regiao': 'Unipam'},
    {'umidade': 60.0, 'temperatura': 30.0, 'dt_criacao': '15/09/23 01:11:18', 'regiao': 'Unipam'},
    {'temperatura': 22, 'umidade': 87.0, 'dt_criacao': '17/09/23 22:13:26', 'regiao': 'Caiçaras'}
]

resultado = []

for dado in dados:
    regiao = dado['regiao']
    novo_dado = {'temperatura': dado.get('temperatura'), 'umidade': dado.get('umidade'), 'dt_criacao': dado.get('dt_criacao')}
    
    # Procurar se a região já está presente na lista resultado
    regiao_existente = next((item for item in resultado if item['regiao'] == regiao), None)
    
    if regiao_existente:
        regiao_existente['dados'].append(novo_dado)
    else:
        resultado.append({'regiao': regiao, 'dados': [novo_dado]})

print(resultado)

In [25]:
lookup = {"$lookup": {"from": "dados_metereologicos",
                        "localField": "_id",
                        "foreignField": "id_regiao",
                        "as": "dadosM"}}
project = {"$project": {'_id':0, 'cidade':0, 'dadosM._id': 0, 'dadosM.id_regiao':0, 'dadosM.regiao':0}}

pipeline = [lookup, project]

results = list(regiao.aggregate(pipeline))
print(results)

resultsFilter = []

for dados in results:
    for maps in dados['dadosM']:
        for key, value in maps.items():
            if (type(value) == Decimal128):
                maps[key] = float(str(maps[key]))
            elif (type(value) == datetime.datetime):
                maps[key] = maps[key].strftime('%d/%m/%y %H:%M:%S')
    resultsFilter.append(dados)

print(resultsFilter)

[{'regiao': 'Unipam', 'dadosM': [{'umidade': 80.0, 'temperatura': 20.0, 'dt_criacao': datetime.datetime(2023, 9, 16, 1, 11, 18, 965000)}, {'umidade': 60.0, 'temperatura': 30.0, 'dt_criacao': datetime.datetime(2023, 9, 15, 1, 11, 18, 965000)}]}, {'regiao': 'Caiçaras', 'dadosM': [{'temperatura': 22, 'umidade': Decimal128('87'), 'dt_criacao': datetime.datetime(2023, 9, 17, 22, 13, 26, 171000)}]}]
[{'regiao': 'Unipam', 'dadosM': [{'umidade': 80.0, 'temperatura': 20.0, 'dt_criacao': '16/09/23 01:11:18'}, {'umidade': 60.0, 'temperatura': 30.0, 'dt_criacao': '15/09/23 01:11:18'}]}, {'regiao': 'Caiçaras', 'dadosM': [{'temperatura': 22, 'umidade': 87.0, 'dt_criacao': '17/09/23 22:13:26'}]}]


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
22

5 days, 0:00:00
